In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! cp /content/drive/MyDrive/Images_resized.tar.gz /content
! tar  x -f Images_resized.tar.gz
! cp /content/drive/MyDrive/MSCOCO_TRAIN_118K.tar.gz /content
! tar  x -f MSCOCO_TRAIN_118K.tar.gz

In [ ]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img,(128,128))
        if img is not None:
            images.append(img)
    return images

images = load_images_from_folder("/content/test2017")

In [ ]:
for i in range(len(images)):
    file_name = "Images_resized/MSCOCO_TEST2017_" + str (i) + ".jpg"
    cv2.imwrite(file_name, images[i])

In [ ]:
%cd /content/drive/MyDrive/colorization_dataSet_folder


In [ ]:
os.listdir("/content/drive/MyDrive/colorization_dataSet_folder/DIV2K_valid_HR")

[]

In [ ]:
import os
import shutil
fs=os.listdir("/content/drive/MyDrive/colorization_dataSet_folder/flower_photos/dandelion")
pd="/content/drive/MyDrive/colorization_dataSet_folder/flower_photos/dandelion"
for f in fs:
    temp_dir=os.path.join(pd,f)

    shutil.move(temp_dir,"/content/drive/MyDrive/colorization_dataSet_folder/DIV2K_train_HR")

In [ ]:
from sklearn.preprocessing import Normalizer

def convertSavedFromBgrToLab(data):
    X_train = np.copy(data)
    X_train = X_train.astype(np.float32)
    res = np.zeros_like(X_train)

    for i in range(X_train.shape[0]):
        res[i] = cv2.cvtColor(X_train[i], cv2.COLOR_BGR2Lab)
    
    print(res[0][0][0])

    res = np.array(res)
    
    mean = np.mean(res, axis = 0)
    std_dev = np.std(res, axis = 0)

    res -= mean
    res /= std_dev
    max = np.max(res, axis=0)
    min = np.min(res, axis=0)

    res -= min
    res /= (max - min)
    
    print(res[0][0][0])

    return res, mean, std_dev, min, max

def convertFromLab2BGR(data):
    X_train = np.copy(data)
    X_train = X_train.astype(np.float32)
    res = np.zeros_like(X_train)
    for i in range(X_train.shape[0]):
        res[i] = cv2.cvtColor(X_train[i], cv2.COLOR_Lab2BGR)
    return res
    





In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import numpy as np


data = np.load("/content/drive/MyDrive/Flower_Images_Data/X_train_128.npy")
print(data.shape)
data = data.reshape(3170, 128,128,3)
print(data.shape)


(3170, 49152)
(3170, 128, 128, 3)


In [ ]:
resized = []
for i in range(data.shape[0]):
    resized.append(cv2.resize(data[i], (64,64)))

resized = np.array(resized)
data = resized

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from keras.models import Model
import matplotlib.pyplot as plt
import cv2
from skimage.color import rgb2lab, lab2rgb
#(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
import skimage.color as converter


y_train_AB, mean, std_dev, min, max = convertSavedFromBgrToLab(data)
X_train_L = np.copy(y_train_AB[:,:,:,0]) 
X_train_L = np.expand_dims(X_train_L, -1)
y_train_AB = y_train_AB[:,:,:,1:]

In [ ]:
y_pred = 0

y_pred = model.predict(X_train_L[0:100])

full_lab_again_predicted = np.concatenate((X_train_L[0:100], y_pred[0:100]), axis=-1)

full_lab_again_predicted = ((full_lab_again_predicted * (max - min)) + min) * std_dev + mean 

RGB_again_predicted = convertFromLab2BGR(full_lab_again_predicted)



full_lab_again_GT = np.concatenate((X_train_L, y_train_AB), axis=-1)

full_lab_again_GT = ((full_lab_again_GT * (max - min)) + min) * std_dev + mean 

RGB_again_GT = convertFromLab2BGR(full_lab_again_GT)


In [ ]:
cv2_imshow(RGB_again_predicted[1]*255)
cv2_imshow(RGB_again_GT[1]*255)

In [ ]:
print(X_train_L[0][0][0])
print(y_train_AB[0][0][0])
print(y_pred[0][0][0])


In [ ]:
import keras.backend as K
import tensorflow as tf
from keras.layers import Input, Conv2D, BatchNormalization, UpSampling2D, Dropout
from keras.models import Model
from keras.regularizers import l2
from keras.utils import multi_gpu_model
from keras.utils import plot_model

# from config import img_rows, img_cols, num_classes, kernel

l2_reg = l2(1e-4)

img_rows = 64 
img_cols = 64
num_classes = 313
kernel = 3

def build_model():
    input_tensor = Input(shape=(img_rows, img_cols, 1))
    x = Conv2D(64, (kernel, kernel), activation='relu', padding='same', name='conv1_1', kernel_initializer="he_normal",
               kernel_regularizer=l2_reg)(input_tensor)
    x = Conv2D(64, (kernel, kernel), activation='relu', padding='same', name='conv1_2', kernel_initializer="he_normal",
               kernel_regularizer=l2_reg, strides=(1, 1))(x)
    # x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, (kernel, kernel), activation='relu', padding='same', name='conv2_1', kernel_initializer="he_normal",
               kernel_regularizer=l2_reg)(x)
    x = Conv2D(128, (kernel, kernel), activation='relu', padding='same', name='conv2_2', kernel_initializer="he_normal",
               kernel_regularizer=l2_reg,
               strides=(2, 2))(x)
    # x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, (kernel, kernel), activation='relu', padding='same', name='conv3_1',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(256, (kernel, kernel), activation='relu', padding='same', name='conv3_2',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(256, (kernel, kernel), activation='relu', padding='same', name='conv3_3', kernel_initializer="he_normal",
               strides=(2, 2))(x)
    # x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    x = Conv2D(512, (kernel, kernel), activation='relu', padding='same', name='conv4_1',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(512, (kernel, kernel), activation='relu', padding='same', name='conv4_2',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(512, (kernel, kernel), activation='relu', padding='same', name='conv4_3',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    # x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    x = Conv2D(512, (kernel, kernel), activation='relu', padding='same', dilation_rate=1, name='conv5_1',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(512, (kernel, kernel), activation='relu', padding='same', dilation_rate=1, name='conv5_2',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(512, (kernel, kernel), activation='relu', padding='same', dilation_rate=1, name='conv5_3',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    # x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    x = Conv2D(512, (kernel, kernel), activation='relu', padding='same', dilation_rate=1, name='conv6_1',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(512, (kernel, kernel), activation='relu', padding='same', dilation_rate=1, name='conv6_2',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(512, (kernel, kernel), activation='relu', padding='same', dilation_rate=1, name='conv6_3',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    # x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    x = Conv2D(256, (kernel, kernel), activation='relu', padding='same', name='conv7_1',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(256, (kernel, kernel), activation='relu', padding='same', name='conv7_2',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(256, (kernel, kernel), activation='relu', padding='same', name='conv7_3',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    # x = Dropout(0.5)(x)
    x = BatchNormalization()(x)


#######
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (kernel, kernel), activation='relu', padding='same', name='conv8_1',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(128, (kernel, kernel), activation='relu', padding='same', name='conv8_2',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    x = Conv2D(128, (kernel, kernel), activation='relu', padding='same', name='conv8_3',
               kernel_initializer="he_normal", kernel_regularizer=l2_reg)(x)
    # x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    # outputs = Conv2D(num_classes, (1, 1), activation='softmax', padding='same', name='pred')(x)
    outputs = Conv2D(2, (1, 1), activation='softmax', padding='same', name='pred')(x)


    model = Model(inputs=input_tensor, outputs=outputs, name="DeepColorizer")
    return model

In [ ]:
import os
%cd /content/

dir_valid = os.listdir("content/Images_resized")
for i in range(len(dir_valid)):
    dir_valid[i] = "content/Images_resized/" + dir_valid[i] 


dir_train = os.listdir("content/MSCOCO_TRAIN2017_RESIZED")
for i in range(len(dir_valid)):
    dir_train[i] = "content/MSCOCO_TRAIN2017_RESIZED/" + dir_train[i] 


gen_valid = generator(dir_valid, dir_valid, batch_size=64)
gen_train = generator(dir_train, dir_train, batch_size=64)

In [ ]:
import keras
ckpt=keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/Flower_Images_Data/flowers_final_2.h5",save_best_only=True)

In [ ]:
model = build_model()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse')
# keras.models.load_model(model,"/content/drive/MyDrive/Flower_Images_Data/model_flowers.h5")

In [ ]:
import cv2

# model.summary()
tf.keras.utils.plot_model(model)
history = model.fit(X_train_L,
                    y_train_AB,
                    batch_size= 128,
                    epochs= 10,
                    validation_split=0.2,
                    callbacks=[ckpt]
                   )
# # print(cv2.imread("content/MSCOCO_TRAIN2017_RESIZED/MSCOCO_TRAIN2017_21670.jpg"))
# history = model.fit(gen_train,
#                     validation_data=gen_valid,
#                     validation_steps=64,
#                     steps_per_epoch=64,
#                     epochs= 100,
#                     callbacks=[ckpt]
#                    )


In [ ]:
from tensorflow import keras
model = keras.models.load_model("/content/drive/MyDrive/Flower_Images_Data/model_for_dev2k_flowers_merge_1.h5")

In [ ]:
keras.models.save_model(model,"model.h5")

In [ ]:
import skimage.transform as rs 
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2 
import numpy as np

img = cv2.imread("/content/image10.jpg")
img = cv2.resize(img, (64,64))
grey = np.copy(img)
cv2_imshow(img)

img = img / 255 
img = img.astype(np.float32)
img = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
# img  = (((img - mean) / std_dev) - min) / (max - min)

X_test_L = np.copy(img[:,:,0])
X_test_L = X_test_L[np.newaxis,:,:,np.newaxis] 


y_test_AB = img[:,:,1:]
#print(X_test_L.shape)
#print(y_test_AB.shape)

y_pred = model.predict(X_test_L)

#######################################

#print(y_pred.shape)
full_lab_again = np.concatenate((X_test_L, y_pred), axis=-1)

# full_lab_again = (full_lab_again* (max - min) + min) * std_dev + mean

#print(full_lab_again.shape)
#print(y_pred[0][0][0])
RGB_again = convertFromLab2BGR(full_lab_again)
# RGB_again = cv2.cvtColor(RGB_again[0]*255, cv2.COLOR_RGB2BGR)

cv2_imshow(RGB_again[0]*255)
grey = cv2.cvtColor(grey, cv2.COLOR_BGR2GRAY)
grey = cv2.resize(grey, (64,64))
colored = cv2.resize(RGB_again[0]*255, (64,64))

cv2.imwrite("flower_11_colored.jpg", colored)
cv2.imwrite("flower_11_grey.jpg", grey)


In [ ]:

print(y_pred.shape)
full_lab_again = np.concatenate((X_test_L, y_pred), axis=-1)
print(full_lab_again.shape)
print(y_pred[0][0][0])
RGB_again = convertFromLab2BGR(full_lab_again)


cv2_imshow(RGB_again[0]*255)



In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip

In [ ]:
!unzip train2017.zip

In [ ]:
import cv2
import os

def load_images_from_folder(folder):
    i = 0
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img,(128,128))
        i+=1
        if (i%1000 == 0):
            print(i)
        if img is not None:
            save_pic = "/content/MSCOCO_TRAIN2017_RESIZED/MSCOCO_TRAIN2017_" + str (i) + ".jpg"
            cv2.imwrite(save_pic, img)
 

load_images_from_folder("/content/Images_resized/train2017")


In [ ]:
!pwd

i = 0
for filename in os.listdir("/content/MSCOCO_TRAIN2017_RESIZED"):
    i +=1

print(i)

In [ ]:

!tar -czvf MSCOCO_TRAIN_118K.tar.gz /content/MSCOCO_TRAIN2017_RESIZED
